In [12]:
import pandas as pd
import numpy as np
import math
import sys

df = pd.read_csv('train.csv')
topk = 10

cantidad_registros = df.shape[0]

usuarios = pd.unique(df["user"])
cantidad_usuarios = len(usuarios)

nuevos_registros = cantidad_usuarios * topk

df2 = df[["continent"]]
df2 = df2.groupby(by=["continent"]).size()
df2 = df2.reset_index()
df2.columns = ["continent", "count-c"]

df2["porc_cont"] = df2["count-c"] / cantidad_registros

df2["nuevas_cont"] = df2["porc_cont"] * nuevos_registros
df2['nuevas_cont'] = df2['nuevas_cont'].apply(round)

df2['porc_cont'] = df2['porc_cont'].apply(lambda x: '%.16f' % x)

df2 = df2.sort_values(
    by=['porc_cont'],
    ascending=[True])

suma = df2["nuevas_cont"].sum()

diferencia = nuevos_registros - suma

cantidad_filas = df2.shape[0]

df2.iat[cantidad_filas-1, 3] = df2.iat[cantidad_filas-1, 3] + diferencia

df2 = df2[["continent", "porc_cont","nuevas_cont"]]

df2.to_csv("porc-cont.csv", index=False,  header=True)

print("\nGuardando... proceso finalizado")


Guardando... proceso finalizado


In [13]:
import pandas as pd
import numpy as np
import sys

df = pd.read_csv('train.csv')
topk = 10

cantidad_registros = df.shape[0]

df2 = df[["user", "genre"]]
df2 = df2.groupby(by=["user", "genre"]).size()
df2 = df2.reset_index()
df2.columns = ["user", "genre", "count-ug"]

df3 = df[["user"]]
df3 = df3.groupby(by=["user"]).size()
df3 = df3.reset_index()
df3.columns = ["user", "count-u"]

df_resultado = pd.merge(df2, df3, how="inner", on=["user"])
df_resultado["porc_ug"] = df_resultado["count-ug"] / df_resultado["count-u"]

df_resultado["nuevas_ug"] = df_resultado["porc_ug"] * topk
df_resultado['nuevas_ug'] = df_resultado['nuevas_ug'].apply(round)
df_resultado.loc[df_resultado["nuevas_ug"] == 0, "nuevas_ug"] = 1

df_resultado['ug'] = df_resultado.agg('{0[user]}-{0[genre]}'.format, axis=1)

df_resultado = df_resultado[["user", "genre", "porc_ug", "nuevas_ug","ug","count-ug","count-u"]]

df_resultado.to_csv("porc-gen.csv", index=False,  header=True)

print("\nGuardando... proceso finalizado")


Guardando... proceso finalizado


In [14]:
import pandas as pd
import logging
import math
from datetime import datetime
import numpy as np
from sklearn.preprocessing import minmax_scale
from pathlib import Path

np.set_printoptions(threshold=3000)
np.set_printoptions(linewidth=100)
np.set_printoptions(edgeitems=10)

now = datetime.now()
print(now)

pd.options.mode.chained_assignment = None  # default='warn'

work_dir = Path('.').resolve()
algorithm =  'vaecf_default'
df = pd.read_csv(algorithm +'.csv')
topk = 10
topn = 1000 

# Normalize rating (0.0-1.0 range)
df['rating'] = minmax_scale(df['rating'])
df = df.to_numpy()
print(df.shape)
cantidad_registros = df.shape[0]
df = np.column_stack((df, np.zeros(cantidad_registros)))
df = np.column_stack((df, np.zeros(cantidad_registros)))

index = [0] * cantidad_registros

for i in range(cantidad_registros):
    base = math.floor(i / topn)
    inicio = base * topn
    index[i] = inicio + topk - 1

col_rating=3
col_pos = 4
base_loss = 7
loss = 8

df[:, base_loss] = df[index, col_rating]
df[:, loss] = (df[:, col_rating] - df[:, base_loss]) / (df[:, base_loss] + 0.0000000000000001)

#Poner en cero (0.0) la perdidas donde la posición sea menor del topk
indexes = np.where(df[:, col_pos] < topk)
df[indexes, loss] = 0

#Eliminar la columna de base-loss
df = np.delete(df,base_loss,1)

df2 = pd.DataFrame(data=df,
                   columns=["id",
                        "user",
                        "item",
                        "rating",
                        "position",
                        "continent",
                        "genre",  
                        "loss"
                        ])

loss_max = min(df2["loss"])
print('loss_max:', loss_max)

df2.to_csv(algorithm +"-loss.csv", index=False,  header=True)

now = datetime.now()
print(now)

print("\nGuardando... proceso finalizado")

2025-01-08 01:19:25.511602


FileNotFoundError: [Errno 2] No such file or directory: 'vaecf_default.csv'

In [10]:
import pandas as pd
import numpy as np
import sys

algorithm = 'vaecf_default'
df = pd.read_csv(algorithm+'-loss.csv')

df_cont = pd.read_csv('porc-cont.csv')
df_genero = pd.read_csv('porc-gen.csv')

df_cont = df_cont[["continent","porc_cont","nuevas_cont"]]
df_genero = df_genero[["user", "genre", "porc_ug", "nuevas_ug"]] 

df2 = pd.merge(df, df_cont, how="left", on=["continent"])
df2 = pd.merge(df2, df_genero, how="left", on=["user", "genre"])

df2 = df2.sort_values(
    by=['porc_cont',"continent", "porc_ug","genre","user", "rating"],
    ascending=[True, True, True, True, True, False])

df2.to_csv(algorithm+"-join.csv", index=False, header=True)

print("\nGuardando... proceso finalizado")


Guardando... proceso finalizado


In [11]:
from re import S
import pandas as pd
import numpy as np
import sys
import logging

#logging.basicConfig(
    #format='%(asctime)s -> %(message)s',
    #level=logging.INFO,
    #handlers=[logging.FileHandler("calibracion.log"),
              #logging.StreamHandler()]
#)

# np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(threshold=3000)
np.set_printoptions(linewidth=100)
np.set_printoptions(edgeitems=10)

col_id = 0
col_user = 1
col_item = 2
col_rating = 3
col_pos = 4
col_cont = 5
col_gen = 6
col_loss = 7
col_porc_cont = 8
nuevas_cont = 9
col_porc_ug = 10
nuevas_ug = 11
col_prioridad = 12
col_cu = 13
col_cug = 14
col_cc = 15
col_fase = 16
col_destino = 17

columns = ['id',
           'user',
           'item',
           'rating',
           'position',
           'continent',
           'genre',
           'loss',
           'porc_cont',
           'nuevas_cont',
           'porc_ug',
           'nuevas_ug',
           'prioridad',
           'cu',
           'cug',
           'cc',
           'Fase',
           'Destino']

tolerancia = 1
algorithm = 'vaecf_default'
df = pd.read_csv(algorithm+'-join.csv')
topk = 10

df['prioridad'] = df['porc_ug'].apply(lambda x: 3 if x <=0.1 else 2)
df['is_max'] = df['rating'].eq((df.loc[df['prioridad'] == 3]).groupby('user')['rating'].transform('max'))
df.loc[df['is_max'] == True,'prioridad'] = 1

df['rating_2'] = df['rating']
df.loc[df['prioridad'] == 1,'rating_2' ] = 1  
df.loc[df['prioridad'] == 2,'rating_2' ] = 1  

df = df.sort_values(
    by=['prioridad','rating_2','porc_cont', "continent", "porc_ug", "genre", "user", "rating"],
    ascending=[True, False, True, True, True, True, True, False])

del df["is_max"]
del df["rating_2"]

df = df.to_numpy()

cantidad_registros = df.shape[0]

logging.info("Realizando configuración básica...")
df = np.column_stack((df, np.zeros(cantidad_registros)))
df = np.column_stack((df, np.zeros(cantidad_registros)))
df = np.column_stack((df, np.zeros(cantidad_registros)))
df = np.column_stack((df, np.zeros(cantidad_registros)))
df = np.column_stack((df,  [' '] * cantidad_registros))

df_cont = pd.read_csv('porc-cont.csv')
df_cont = df_cont.to_numpy()
cantidad_registros_cont = df_cont.shape[0]

df_gen = pd.read_csv('porc-gen.csv')
df_gen = df_gen.to_numpy()
cantidad_registros_gen = df_gen.shape[0]

df[:, col_loss] = abs(df[:, col_loss])  # Convertir en positivas las perdidas

usuarios = np.unique(df[:, col_user])
contador_usuarios = dict(zip(usuarios, np.zeros(len(usuarios))))

ug_array = np.unique(df_gen[:, 4])
contador_ug = dict(zip(ug_array, np.zeros(len(ug_array))))

continentes = np.unique(df[:, col_cont])
contador_continentes = dict(zip(continentes, np.zeros(len(continentes))))

nuevas_filas_ug = dict(zip(df_gen[:, 4].tolist(), df_gen[:, 3].tolist()))

nuevas_filas_cont = dict(zip(df_cont[:, 0].tolist(), df_cont[:, 2].tolist()))

x=1
cantidad_usuarios = len(usuarios)

logging.info("Fase 1:")
for i in range(cantidad_registros):
    row = df[i]
    
    if(row[col_loss] <= tolerancia):
        # if(True):
        usuario = row[col_user]
        genero = row[col_gen]
        ug = str(usuario) + "-" + genero
        continente = row[col_cont]

        if (ug in nuevas_filas_ug) and (continente in nuevas_filas_cont):
            cu = contador_usuarios[usuario] + 1
            cug = contador_ug[ug] + 1
            cc = contador_continentes[continente] + 1

            if(nuevas_filas_ug[ug] >= cug and
                    nuevas_filas_cont[continente] >= cc and
                    topk >= cu):

                row[col_cu] = cu
                row[col_cug] = cug
                row[col_cc] = cc

                row[col_fase] = 1  # Fase 1
                row[col_destino] = str(usuario) + "-" + genero + "-" + continente

                contador_usuarios[usuario] = cu
                contador_ug[ug] = cug
                contador_continentes[continente] = cc
                
                df[i] = row
    
    if(i % 10000 == 0):
        logging.info(str(i) + " / " + str(cantidad_registros) + ": " +
                     str(round(i/cantidad_registros*100, 2)) + "%")

logging.info("Fase 2:")
for i in range(cantidad_registros):
    row = df[i]
    
    if(row[col_loss] <= tolerancia and row[col_fase] == 0):
        # if(True):
        usuario = row[col_user]
        genero = row[col_gen]
        ug = str(usuario) + "-" + genero
        continente = row[col_cont]

        if (continente in nuevas_filas_cont):
            cu = contador_usuarios[usuario] + 1
            cc = contador_continentes[continente] + 1

            if(nuevas_filas_cont[continente] >= cc and topk >= cu):
                row[col_cu] = cu
                row[col_cug] = cug
                row[col_cc] = cc

                row[col_fase] = 2  # Fase 2
                row[col_destino] = str(usuario) + "-" + genero + "-" + continente

                contador_usuarios[usuario] = cu
                contador_ug[ug] = cug
                contador_continentes[continente] = cc

                df[i] = row

    if(i % 10000 == 0):
        logging.info(str(i) + " / " + str(cantidad_registros) + ": " +
                     str(round(i/cantidad_registros*100, 2)) + "%")

logging.info("Fase 3:")
for i in range(cantidad_registros):
    row = df[i]

    if(row[col_loss] <= tolerancia and row[col_fase] == 0):
        # if(True):
        usuario = row[col_user]
        genero = row[col_gen]
        ug = str(usuario) + "-" + genero
        continente = row[col_cont]

        if (True):
            cu = contador_usuarios[usuario] + 1

            if(topk >= cu):
                row[col_cu] = cu
                
                row[col_fase] = 3  # Fase 3
                row[col_destino] = str(usuario) + "-" + genero + "-" + continente
                
                contador_usuarios[usuario] = cu
                
                df[i] = row

    if(i % 10000 == 0):
        logging.info(str(i) + " / " + str(cantidad_registros) + ": " +
                     str(round(i/cantidad_registros*100, 2)) + "%")

indexes = np.where(df[:, col_fase]  == 0)
df = np.delete(df, indexes, 0)
  
    
logging.info(contador_usuarios)

logging.info("Ordenando...")

df2 = pd.DataFrame(data=df, columns=columns)

df2 = df2.sort_values(
   by=['user', 'rating'],
   ascending=[True, False])

df = df2.to_numpy()

logging.info("Guardando resultados...")
df2 = pd.DataFrame(data=df, columns=columns)

df2['porc_cont'] = df2['porc_cont'].apply(
    lambda x: '%.16f' % x)

df2['porc_ug'] = df2['porc_ug'].apply(
    lambda x: '%.16f' % x)

df2['loss'] = df2['loss'].apply(
    lambda x: '%.16f' % x)

df2.to_csv(algorithm +'-'+str(round(tolerancia*100))+'.csv', index=False, header=True)

print("\nGuardando... proceso finalizado")


Guardando... proceso finalizado
